# FRED Economic Data Analysis

This notebook demonstrates how to retrieve and analyze economic data from the Federal Reserve Economic Data (FRED) API using LangChain for investment research.

## Features

- Economic indicator retrieval and analysis
- GDP, inflation, unemployment data processing
- Interest rates and monetary policy analysis
- LangChain integration for economic insights
- Time series analysis and trend detection
- Investment implications assessment

## Prerequisites

- FRED API key (free from https://fred.stlouisfed.org/docs/api/api_key.html)
- Required Python packages: requests, pandas, langchain, python-dotenv


# Install required packages


In [17]:
%pip install requests pandas langchain langchain-text-splitters python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Install required packages


In [18]:
# Import required libraries
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import os
from typing import Dict, List, Optional, Any
import time

# LangChain imports
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

# Environment variables
from dotenv import load_dotenv
load_dotenv()

print("Libraries imported successfully!")
print(f"Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Libraries imported successfully!
Analysis started at: 2025-09-20 22:56:48


## Configuration and Setup

Configure API credentials and analysis parameters.


In [19]:
# Configuration
FRED_API_KEY = os.getenv('FRED_API_KEY')
if not FRED_API_KEY:
    print("Warning: FRED_API_KEY not found in environment variables.")
    print("Please set your FRED API key in the .env file.")
    print("Get your free API key from: https://fred.stlouisfed.org/docs/api/api_key.html")
else:
    print("FRED API key loaded successfully!")

# FRED API configuration
FRED_BASE_URL = "https://api.stlouisfed.org/fred"
REQUEST_TIMEOUT = 30
RATE_LIMIT_DELAY = 0.5  # Delay between requests

# Analysis parameters
ANALYSIS_PERIOD_YEARS = 5  # Historical data period
end_date = datetime.now()
start_date = end_date - timedelta(days=ANALYSIS_PERIOD_YEARS * 365)

print(f"Analysis period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

# Key economic indicators to analyze
ECONOMIC_INDICATORS = {
    'GDP': 'GDP',  # Gross Domestic Product
    'UNEMPLOYMENT': 'UNRATE',  # Unemployment Rate
    'INFLATION': 'CPIAUCSL',  # Consumer Price Index
    'FED_FUNDS_RATE': 'FEDFUNDS',  # Federal Funds Rate
    'TREASURY_10Y': 'GS10',  # 10-Year Treasury Rate
    'TREASURY_2Y': 'GS2',  # 2-Year Treasury Rate
    'CONSUMER_SENTIMENT': 'UMCSENT',  # Consumer Sentiment
    'INDUSTRIAL_PRODUCTION': 'INDPRO'  # Industrial Production Index
}

print(f"Configured {len(ECONOMIC_INDICATORS)} economic indicators for analysis")

FRED API key loaded successfully!
Analysis period: 2020-09-21 to 2025-09-20
Configured 8 economic indicators for analysis


## FRED API Client Class

Professional client for accessing FRED economic data with comprehensive error handling and data processing capabilities.


In [20]:
class FREDEconomicClient:
    """
    Professional client for FRED Economic Data API with comprehensive data processing.
    
    Provides methods for retrieving economic indicators, analyzing trends,
    and generating investment-relevant insights from Federal Reserve data.
    """
    
    def __init__(self, api_key: str, base_url: str = FRED_BASE_URL):
        """Initialize FRED client with API credentials."""
        self.api_key = api_key
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Investment-Research-Platform/1.0'
        })
    
    def get_series_data(self, series_id: str, start_date: str, end_date: str) -> Dict[str, Any]:
        """
        Retrieve time series data for a specific economic indicator.
        
        Args:
            series_id: FRED series identifier
            start_date: Start date in YYYY-MM-DD format
            end_date: End date in YYYY-MM-DD format
            
        Returns:
            Dictionary containing series data and metadata
        """
        try:
            # Build API request
            url = f"{self.base_url}/series/observations"
            params = {
                'series_id': series_id,
                'api_key': self.api_key,
                'file_type': 'json',
                'observation_start': start_date,
                'observation_end': end_date,
                'sort_order': 'desc',
                'limit': 10000  # Maximum observations
            }
            
            # Make request with rate limiting
            time.sleep(RATE_LIMIT_DELAY)
            response = self.session.get(url, params=params, timeout=REQUEST_TIMEOUT)
            response.raise_for_status()
            
            data = response.json()
            
            if 'observations' not in data:
                return {'observations': [], 'series_id': series_id, 'error': 'No data'}
            
            # Process observations
            observations = []
            for obs in data['observations']:
                if obs['value'] != '.':
                    try:
                        observations.append({
                            'date': obs['date'],
                            'value': float(obs['value'])
                        })
                    except (ValueError, TypeError):
                        continue
            
            return {
                'series_id': series_id,
                'observations': observations,
                'count': len(observations),
                'latest_value': observations[0]['value'] if observations else None,
                'latest_date': observations[0]['date'] if observations else None
            }
            
        except requests.exceptions.RequestException as e:
            return {'series_id': series_id, 'error': str(e), 'observations': []}
        except Exception as e:
            return {'series_id': series_id, 'error': str(e), 'observations': []}
    
    def get_series_info(self, series_id: str) -> Dict[str, Any]:
        """Get metadata information for a FRED series."""
        try:
            url = f"{self.base_url}/series"
            params = {
                'series_id': series_id,
                'api_key': self.api_key,
                'file_type': 'json'
            }
            
            time.sleep(RATE_LIMIT_DELAY)
            response = self.session.get(url, params=params, timeout=REQUEST_TIMEOUT)
            response.raise_for_status()
            
            data = response.json()
            
            if 'seriess' in data and data['seriess']:
                series_info = data['seriess'][0]
                return {
                    'id': series_info.get('id'),
                    'title': series_info.get('title'),
                    'units': series_info.get('units'),
                    'frequency': series_info.get('frequency'),
                    'last_updated': series_info.get('last_updated'),
                    'notes': series_info.get('notes', '')
                }
            
            return {'error': 'Series not found'}
            
        except Exception as e:
            return {'error': str(e)}
    
    def calculate_trends(self, observations: List[Dict]) -> Dict[str, Any]:
        """Calculate trend analysis for time series data."""
        if len(observations) < 2:
            return {'error': 'Insufficient data for trend analysis'}
        
        try:
            # Sort by date (oldest first for trend calculation)
            sorted_obs = sorted(observations, key=lambda x: x['date'])
            values = [obs['value'] for obs in sorted_obs]
            
            # Basic trend calculations
            current_value = values[-1]
            previous_value = values[-2] if len(values) > 1 else values[0]
            start_value = values[0]
            
            # Period changes
            period_change = current_value - previous_value
            period_change_pct = ((current_value - previous_value) / previous_value * 100) if previous_value != 0 else 0
            
            # Total change over period
            total_change = current_value - start_value
            total_change_pct = ((current_value - start_value) / start_value * 100) if start_value != 0 else 0
            
            # Volatility (standard deviation)
            mean_value = sum(values) / len(values)
            variance = sum((x - mean_value) ** 2 for x in values) / len(values)
            volatility = variance ** 0.5
            
            # Trend direction
            recent_values = values[-12:] if len(values) >= 12 else values  # Last 12 periods
            if len(recent_values) >= 3:
                trend_slope = (recent_values[-1] - recent_values[0]) / len(recent_values)
                if trend_slope > volatility * 0.1:
                    trend_direction = 'Rising'
                elif trend_slope < -volatility * 0.1:
                    trend_direction = 'Falling'
                else:
                    trend_direction = 'Stable'
            else:
                trend_direction = 'Insufficient data'
            
            return {
                'current_value': current_value,
                'previous_value': previous_value,
                'period_change': period_change,
                'period_change_pct': period_change_pct,
                'total_change': total_change,
                'total_change_pct': total_change_pct,
                'volatility': volatility,
                'trend_direction': trend_direction,
                'mean_value': mean_value,
                'min_value': min(values),
                'max_value': max(values),
                'data_points': len(values)
            }
            
        except Exception as e:
            return {'error': f'Trend calculation error: {e}'}

# Initialize FRED client
if FRED_API_KEY:
    fred_client = FREDEconomicClient(FRED_API_KEY)
    print("FRED client initialized successfully")
else:
    print("FRED API key required")
    fred_client = None

FRED client initialized successfully


## Economic Data Retrieval

Fetch and process key economic indicators for investment analysis.


In [21]:
if fred_client:
    print("Retrieving economic indicators...")
    
    # Store all economic data
    economic_data = {}
    economic_metadata = {}
    
    # Retrieve data for each indicator
    for indicator_name, series_id in ECONOMIC_INDICATORS.items():
        # Get series metadata
        metadata = fred_client.get_series_info(series_id)
        economic_metadata[indicator_name] = metadata
        
        # Get series data
        start_str = start_date.strftime('%Y-%m-%d')
        end_str = end_date.strftime('%Y-%m-%d')
        
        data = fred_client.get_series_data(series_id, start_str, end_str)
        economic_data[indicator_name] = data
        
        # Display summary for successful data retrieval
        if 'error' not in data and data['observations']:
            latest_value = data['latest_value']
            latest_date = data['latest_date']
            count = data['count']
            print(f"{indicator_name}: {latest_value} ({latest_date}) - {count} data points")
    
    print("Data retrieval completed")
    
else:
    print("FRED client not available - please configure API key")

Retrieving economic indicators...
GDP: 30353.902 (2025-04-01) - 20 data points
GDP: 30353.902 (2025-04-01) - 20 data points
UNEMPLOYMENT: 4.3 (2025-08-01) - 60 data points
UNEMPLOYMENT: 4.3 (2025-08-01) - 60 data points
INFLATION: 323.364 (2025-08-01) - 60 data points
INFLATION: 323.364 (2025-08-01) - 60 data points
FED_FUNDS_RATE: 4.33 (2025-08-01) - 60 data points
FED_FUNDS_RATE: 4.33 (2025-08-01) - 60 data points
TREASURY_10Y: 4.26 (2025-08-01) - 60 data points
TREASURY_10Y: 4.26 (2025-08-01) - 60 data points
TREASURY_2Y: 3.7 (2025-08-01) - 60 data points
TREASURY_2Y: 3.7 (2025-08-01) - 60 data points
CONSUMER_SENTIMENT: 61.7 (2025-07-01) - 59 data points
CONSUMER_SENTIMENT: 61.7 (2025-07-01) - 59 data points
INDUSTRIAL_PRODUCTION: 103.9203 (2025-08-01) - 60 data points
Data retrieval completed
INDUSTRIAL_PRODUCTION: 103.9203 (2025-08-01) - 60 data points
Data retrieval completed


## Economic Trend Analysis

Analyze trends and calculate key metrics for investment decision making.


In [22]:
if fred_client and 'economic_data' in locals():
    print("Analyzing economic trends...")
    
    # Store trend analysis results
    trend_analysis = {}
    
    # Analyze each economic indicator
    for indicator_name, data in economic_data.items():
        if 'error' not in data and data['observations']:
            # Calculate trends
            trends = fred_client.calculate_trends(data['observations'])
            trend_analysis[indicator_name] = trends
            
            if 'error' not in trends:
                current = trends['current_value']
                period_change_pct = trends['period_change_pct']
                total_change_pct = trends['total_change_pct']
                trend_direction = trends['trend_direction']
                
                print(f"{indicator_name}: {current:.2f} ({period_change_pct:+.1f}%, {trend_direction})")
                
                # Investment implications
                if indicator_name == 'GDP':
                    if period_change_pct > 2:
                        implication = "Strong economic growth - favorable for equities"
                    elif period_change_pct < 0:
                        implication = "Economic contraction - consider defensive assets"
                    else:
                        implication = "Moderate growth - balanced investment approach"
                
                elif indicator_name == 'UNEMPLOYMENT':
                    if trends['current_value'] < 4:
                        implication = "Low unemployment - potential inflation pressure"
                    elif trends['current_value'] > 7:
                        implication = "High unemployment - economic stress signals"
                    else:
                        implication = "Normal unemployment levels"
                
                elif indicator_name == 'INFLATION':
                    if period_change_pct > 3:
                        implication = "High inflation - consider inflation-protected assets"
                    elif period_change_pct < 1:
                        implication = "Low inflation - growth assets may benefit"
                    else:
                        implication = "Moderate inflation levels"
                
                elif indicator_name == 'FED_FUNDS_RATE':
                    if trend_direction == 'Rising':
                        implication = "Rising rates - favor financial sector, consider duration risk"
                    elif trend_direction == 'Falling':
                        implication = "Falling rates - supportive for growth and real estate"
                    else:
                        implication = "Stable rates - current monetary policy stance maintained"
                
                elif indicator_name in ['TREASURY_10Y', 'TREASURY_2Y']:
                    if trends['current_value'] > 4:
                        implication = "High yields - attractive for fixed income investors"
                    elif trends['current_value'] < 2:
                        implication = "Low yields - search for yield in alternative assets"
                    else:
                        implication = "Moderate yield environment"
                
                else:
                    if trend_direction == 'Rising':
                        implication = "Positive trend - monitor for investment opportunities"
                    elif trend_direction == 'Falling':
                        implication = "Declining trend - exercise caution"
                    else:
                        implication = "Stable conditions"
                
                print(f"  → {implication}")
            
            else:
                trend_analysis[indicator_name] = {'error': trends['error']}
        
        else:
            trend_analysis[indicator_name] = {'error': 'No data available'}
    
    print("Trend analysis completed")
    
else:
    print("Economic data not available for trend analysis")

Analyzing economic trends...
GDP: 30353.90 (+1.3%, Rising)
  → Moderate growth - balanced investment approach
UNEMPLOYMENT: 4.30 (+2.4%, Stable)
  → Normal unemployment levels
INFLATION: 323.36 (+0.4%, Stable)
  → Low inflation - growth assets may benefit
FED_FUNDS_RATE: 4.33 (+0.0%, Stable)
  → Stable rates - current monetary policy stance maintained
TREASURY_10Y: 4.26 (-3.0%, Stable)
  → High yields - attractive for fixed income investors
TREASURY_2Y: 3.70 (-4.6%, Stable)
  → Moderate yield environment
CONSUMER_SENTIMENT: 61.70 (+1.6%, Stable)
  → Stable conditions
INDUSTRIAL_PRODUCTION: 103.92 (+0.1%, Stable)
  → Stable conditions
Trend analysis completed


## LangChain Economic Analysis

Use LangChain to process economic data and generate comprehensive investment insights.


In [23]:
if fred_client and 'economic_data' in locals() and 'trend_analysis' in locals():
    print("Creating economic analysis document...")
    
    # Compile comprehensive economic analysis
    economic_report = f"""
FEDERAL RESERVE ECONOMIC DATA ANALYSIS
Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Analysis Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}

ECONOMIC INDICATORS SUMMARY:
"""
    
    # Add each indicator analysis
    for indicator_name, trends in trend_analysis.items():
        if 'error' not in trends:
            metadata = economic_metadata.get(indicator_name, {})
            
            economic_report += f"""

{indicator_name} ({metadata.get('title', 'No description')}):
Current Value: {trends['current_value']:.2f} {metadata.get('units', '')}
Period Change: {trends['period_change']:+.2f} ({trends['period_change_pct']:+.1f}%)
Total Change: {trends['total_change_pct']:+.1f}%
Trend Direction: {trends['trend_direction']}
Volatility: {trends['volatility']:.2f}
Data Points: {trends['data_points']}
"""
        else:
            economic_report += f"""

{indicator_name}: Data unavailable - {trends['error']}
"""
    
    # Create LangChain document
    doc = Document(page_content=economic_report)
    
    # Split document for processing
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=150
    )
    
    chunks = text_splitter.split_documents([doc])
    
    print(f"Economic analysis document: {len(economic_report)} characters, {len(chunks)} chunks")
    
    # Economic environment assessment
    print("Economic environment assessment:")
    
    # Collect economic signals
    growth_signals = []
    inflation_signals = []
    monetary_signals = []
    risk_signals = []
    
    for indicator_name, trends in trend_analysis.items():
        if 'error' not in trends:
            # GDP signals
            if indicator_name == 'GDP':
                if trends['trend_direction'] == 'Rising':
                    growth_signals.append(f"GDP trending upward ({trends['period_change_pct']:+.1f}%)")
                elif trends['trend_direction'] == 'Falling':
                    growth_signals.append(f"GDP declining ({trends['period_change_pct']:+.1f}%)")
                else:
                    growth_signals.append(f"GDP stable ({trends['period_change_pct']:+.1f}%)")
            
            # Unemployment signals
            elif indicator_name == 'UNEMPLOYMENT':
                if trends['current_value'] < 4:
                    growth_signals.append(f"Low unemployment at {trends['current_value']:.1f}%")
                    inflation_signals.append("Tight labor market may pressure wages")
                elif trends['current_value'] > 7:
                    risk_signals.append(f"High unemployment at {trends['current_value']:.1f}%")
                
            # Inflation signals
            elif indicator_name == 'INFLATION':
                if trends['trend_direction'] == 'Rising':
                    inflation_signals.append(f"Inflation rising ({trends['period_change_pct']:+.1f}%)")
                elif trends['current_value'] > 3:
                    inflation_signals.append(f"Elevated inflation at {trends['current_value']:.1f}%")
                else:
                    inflation_signals.append(f"Inflation contained at {trends['current_value']:.1f}%")
            
            # Interest rate signals
            elif indicator_name == 'FED_FUNDS_RATE':
                if trends['trend_direction'] == 'Rising':
                    monetary_signals.append(f"Fed tightening policy (rate: {trends['current_value']:.2f}%)")
                elif trends['trend_direction'] == 'Falling':
                    monetary_signals.append(f"Fed easing policy (rate: {trends['current_value']:.2f}%)")
                else:
                    monetary_signals.append(f"Fed holding rates steady at {trends['current_value']:.2f}%")
            
            # Treasury yield signals
            elif indicator_name in ['TREASURY_10Y', 'TREASURY_2Y']:
                if trends['current_value'] > 4:
                    monetary_signals.append(f"{indicator_name} yields elevated at {trends['current_value']:.2f}%")
                elif trends['current_value'] < 2:
                    monetary_signals.append(f"{indicator_name} yields low at {trends['current_value']:.2f}%")
    
    # Display economic signals summary
    print(f"Growth indicators: {len(growth_signals)}")
    print(f"Inflation indicators: {len(inflation_signals)}")
    print(f"Monetary policy indicators: {len(monetary_signals)}")
    print(f"Risk indicators: {len(risk_signals)}")
    
    # Overall economic assessment
    growth_score = len([s for s in growth_signals if 'upward' in s or 'Low unemployment' in s])
    risk_score = len(risk_signals) + len([s for s in inflation_signals if 'rising' in s or 'Elevated' in s])
    
    if growth_score > risk_score:
        economic_outlook = "EXPANSIONARY - Economic growth conditions favorable"
        investment_theme = "Risk-on environment supports growth assets"
    elif risk_score > growth_score + 1:
        economic_outlook = "CONTRACTIONARY - Economic headwinds present"
        investment_theme = "Defensive positioning recommended"
    else:
        economic_outlook = "TRANSITIONAL - Mixed economic signals"
        investment_theme = "Balanced approach with selective opportunities"
    
    print(f"Economic Outlook: {economic_outlook}")
    print(f"Investment Theme: {investment_theme}")
    
    # Sector implications summary
    sector_implications = []
    if any('tightening' in s for s in monetary_signals):
        sector_implications.extend(["Financials: Benefit from rising rates", "REITs: Pressure from higher rates"])
    elif any('easing' in s for s in monetary_signals):
        sector_implications.extend(["Growth Stocks: Benefit from lower rates", "REITs: Attractive in low-rate environment"])
    
    if any('rising' in s for s in inflation_signals):
        sector_implications.extend(["Commodities: Inflation hedge potential", "TIPS: Inflation-protected bonds attractive"])
    
    if sector_implications:
        print("Key sector implications:")
        for implication in sector_implications[:3]:  # Show top 3
            print(f"  {implication}")
    
    print("Economic analysis completed")
    
else:
    print("Economic data not available for comprehensive analysis")

Creating economic analysis document...
Economic analysis document: 1859 characters, 2 chunks
Economic environment assessment:
Growth indicators: 1
Inflation indicators: 1
Monetary policy indicators: 2
Risk indicators: 0
Economic Outlook: TRANSITIONAL - Mixed economic signals
Investment Theme: Balanced approach with selective opportunities
Economic analysis completed


## Data Export and Integration

Export economic analysis results for integration with investment research platform.


In [24]:
if 'economic_data' in locals() and 'trend_analysis' in locals():
    print("Exporting economic analysis results...")
    
    # Create data directory if it doesn't exist
    import os
    data_dir = "data"
    os.makedirs(data_dir, exist_ok=True)
    
    # Create comprehensive export dataset
    export_data = {
        'analysis_timestamp': datetime.now().isoformat(),
        'analysis_period': {
            'start_date': start_date.strftime('%Y-%m-%d'),
            'end_date': end_date.strftime('%Y-%m-%d'),
            'period_years': ANALYSIS_PERIOD_YEARS
        },
        'economic_indicators': {},
        'trend_analysis': {},
        'metadata': {}
    }
    
    # Export economic data and trends
    for indicator_name in ECONOMIC_INDICATORS.keys():
        if indicator_name in economic_data:
            data = economic_data[indicator_name]
            trends = trend_analysis.get(indicator_name, {})
            metadata = economic_metadata.get(indicator_name, {})
            
            export_data['economic_indicators'][indicator_name] = {
                'series_id': data.get('series_id', ''),
                'latest_value': data.get('latest_value'),
                'latest_date': data.get('latest_date'),
                'data_points': data.get('count', 0),
                'recent_data': data.get('observations', [])[:10] if 'observations' in data else []
            }
            
            export_data['trend_analysis'][indicator_name] = trends
            export_data['metadata'][indicator_name] = metadata
    
    # Add analysis insights
    if 'economic_outlook' in locals():
        export_data['analysis_summary'] = {
            'economic_outlook': economic_outlook,
            'investment_theme': investment_theme,
            'growth_signals': growth_signals if 'growth_signals' in locals() else [],
            'inflation_signals': inflation_signals if 'inflation_signals' in locals() else [],
            'monetary_signals': monetary_signals if 'monetary_signals' in locals() else [],
            'risk_signals': risk_signals if 'risk_signals' in locals() else []
        }
    
    # Export to JSON
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    json_filename = os.path.join(data_dir, f"fred_economic_analysis_{timestamp}.json")
    
    try:
        with open(json_filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        print(f"Economic analysis exported to: {json_filename}")
    except Exception as e:
        print(f"Error exporting JSON: {e}")
    
    # Create CSV for time series data
    try:
        csv_data = []
        for indicator_name, data in economic_data.items():
            if 'observations' in data and data['observations']:
                for obs in data['observations'][:50]:  # Last 50 observations
                    csv_data.append({
                        'indicator': indicator_name,
                        'series_id': data.get('series_id', ''),
                        'date': obs['date'],
                        'value': obs['value'],
                        'units': economic_metadata.get(indicator_name, {}).get('units', '')
                    })
        
        if csv_data:
            df = pd.DataFrame(csv_data)
            csv_filename = os.path.join(data_dir, f"fred_economic_data_{timestamp}.csv")
            df.to_csv(csv_filename, index=False)
            print(f"Time series data exported to: {csv_filename}")
            print(f"Export summary: {len(ECONOMIC_INDICATORS)} indicators, {len(csv_data)} records")
        
    except Exception as e:
        print(f"Error creating CSV export: {e}")
    
    # Integration recommendations
    if 'economic_outlook' in locals():
        print(f"Economic outlook: {economic_outlook}")
        if 'EXPANSIONARY' in economic_outlook:
            print("  Portfolio: Increase allocation to growth assets")
        elif 'CONTRACTIONARY' in economic_outlook:
            print("  Portfolio: Increase defensive allocations")
        else:
            print("  Portfolio: Maintain balanced allocation")
    
    print("Economic data export completed")
    
else:
    print("No economic data available for export")

Exporting economic analysis results...
Economic analysis exported to: data\fred_economic_analysis_20250920_225657.json
Time series data exported to: data\fred_economic_data_20250920_225657.csv
Export summary: 8 indicators, 370 records
Economic outlook: TRANSITIONAL - Mixed economic signals
  Portfolio: Maintain balanced allocation
Economic data export completed


In [25]:
# FRED Economic Analysis Session Summary
print("FRED Economic Data Analysis Completed!")
print(f"Session end time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Summary of analysis performed
analysis_summary = []

if 'economic_data' in locals():
    successful_indicators = sum(1 for data in economic_data.values() if 'error' not in data)
    analysis_summary.append(f"Economic indicators: {successful_indicators}/{len(ECONOMIC_INDICATORS)} retrieved")

if 'trend_analysis' in locals():
    successful_trends = sum(1 for trends in trend_analysis.values() if 'error' not in trends)
    analysis_summary.append(f"Trend analysis: {successful_trends} indicators analyzed")

if 'economic_outlook' in locals():
    analysis_summary.append(f"Economic assessment: {economic_outlook}")

if 'economic_report' in locals():
    analysis_summary.append(f"LangChain document: {len(economic_report)} characters")

print("Analysis components completed:")
for item in analysis_summary:
    print(f"  {item}")

# Key economic insights
if 'trend_analysis' in locals():
    print("Key economic insights:")
    
    # GDP status
    if 'GDP' in trend_analysis and 'error' not in trend_analysis['GDP']:
        gdp_trend = trend_analysis['GDP']['trend_direction']
        gdp_change = trend_analysis['GDP']['period_change_pct']
        print(f"  GDP: {gdp_trend} ({gdp_change:+.1f}%)")
    
    # Inflation status
    if 'INFLATION' in trend_analysis and 'error' not in trend_analysis['INFLATION']:
        inflation_value = trend_analysis['INFLATION']['current_value']
        inflation_trend = trend_analysis['INFLATION']['trend_direction']
        print(f"  Inflation: {inflation_value:.1f}% ({inflation_trend})")
    
    # Interest rates status
    if 'FED_FUNDS_RATE' in trend_analysis and 'error' not in trend_analysis['FED_FUNDS_RATE']:
        fed_rate = trend_analysis['FED_FUNDS_RATE']['current_value']
        fed_trend = trend_analysis['FED_FUNDS_RATE']['trend_direction']
        print(f"  Fed Funds Rate: {fed_rate:.2f}% ({fed_trend})")
    
    # Unemployment status
    if 'UNEMPLOYMENT' in trend_analysis and 'error' not in trend_analysis['UNEMPLOYMENT']:
        unemployment = trend_analysis['UNEMPLOYMENT']['current_value']
        unemployment_trend = trend_analysis['UNEMPLOYMENT']['trend_direction']
        print(f"  Unemployment: {unemployment:.1f}% ({unemployment_trend})")

print("For different analysis periods:")
print("  1. Update ANALYSIS_PERIOD_YEARS variable")
print("  2. Re-run all analysis cells")

print("Ready for integration with other financial analysis")
print("FRED economic analysis framework established successfully")

FRED Economic Data Analysis Completed!
Session end time: 2025-09-20 22:56:57
Analysis components completed:
  Economic indicators: 8/8 retrieved
  Trend analysis: 8 indicators analyzed
  Economic assessment: TRANSITIONAL - Mixed economic signals
  LangChain document: 1859 characters
Key economic insights:
  GDP: Rising (+1.3%)
  Inflation: 323.4% (Stable)
  Fed Funds Rate: 4.33% (Stable)
  Unemployment: 4.3% (Stable)
For different analysis periods:
  1. Update ANALYSIS_PERIOD_YEARS variable
  2. Re-run all analysis cells
Ready for integration with other financial analysis
FRED economic analysis framework established successfully
